# Final C: Visual servoing control

For your final, you will be implementing a visual servoing controller with several variants, and testing its behavior under different sensor and object movement conditions.  The robot is given an eye-in-hand camera, and an object pose detector is assumed given to you.

A basic implementation of the HW4 control scheme is given to you.  In this scheme, the error signal is only the  translation of the object.  The link keeps its orientation fixed regardless of the object orientation.

In [55]:
%load_ext autoreload
%autoreload 2

#these might be useful
from klampt import *
from klampt.math import so3,se3,vectorops
from IPython.display import display,clear_output
import time
import ipywidgets as widgets
import ipyklampt
import numpy as np
import math

world = WorldModel()
fn = "data/ur5Cart.xml"
res = world.readFile(fn)
robot = world.robot(0)

#load the object
objfn = "data/objects/cylinder.obj"
#use this if you want to test how well your controller handles object rotation about z...
#objfn = "data/objects/block_small.obj"
res = world.readFile(objfn)
obj = world.rigidObject(0)
Tobj_orig = (so3.identity(),[0.75,0.21,1.2])
#FOR TESTING: start out out of the camera's FOV
#Tobj_orig = (so3.identity(),[0.75,-0.4,1.2])
obj.setTransform(*Tobj_orig)

#set the home configuration
qhome  = robot.getConfig()
qhome[1] = 2.3562
qhome[2] = -math.pi/3.0
qhome[3] = math.pi*2/3.0
qhome[4] = -0.9
qhome[5] = math.pi/2.0
robot.setConfig(qhome)

kvis = ipyklampt.KlamptWidget(world)
display(kvis)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


KlamptWidget(scene={u'object': {u'matrix': [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1], u'uuid': u'67c4e…

In [56]:
cameraLink = robot.link(7)
cameraXform = ([0,-1,0,0,0,-1,1,0,0],[0.05,0,-0.03])

kvis.add_xform("camera",length=0.1)
kvis.set_transform("camera",*se3.mul(cameraLink.getTransform(),cameraXform))

def setConfig(q):
    robot.setConfig(q)
    kvis.set_transform("camera",*se3.mul(cameraLink.getTransform(),cameraXform))
    kvis.update()

In [57]:
import random

sensorNoiseRotation = 0.0
sensorNoiseTranslation = 0.0
#FOR TESTING: add noise to the camera estimate
#sensorNoiseRotation = 0.1
#sensorNoiseTranslation = 0.02
sensorFOV = math.radians(90.0)
sensorW = 640
sensorH = 480

def emulateSensor():
    """The sensor either produces None if the object is not seen, or otherwise a rigid transform."""
    camWorldXform = se3.mul(cameraLink.getTransform(),cameraXform)
    objXform = obj.getTransform()
    camXform = se3.mul(se3.inv(camWorldXform),objXform)
    x,y,z = camXform[1]
    slope = math.tan(sensorFOV*0.5)
    if z < 0.1:
        #minimum distance
        return None
    if abs(x) > z*slope:
        #out of horizontal FOV
        return None
    if abs(y)*sensorW/sensorH > z*slope:
        #out of vertical FOV
        return None
    
    eR = [random.gauss(0,sensorNoiseRotation*z) for i in range(3)]
    et = [random.gauss(0,sensorNoiseTranslation*z) for i in range(3)]
    return se3.mul(camXform,(so3.from_rotation_vector(eR),et))

print "Emulated object pose:",emulateSensor()

Emulated object pose: ([-1.7754899901929982e-09, 0.14666656540331252, 0.9891859878671934, 0.9999999999999998, 0.0, 1.7949000612323396e-09, 2.632518272231427e-10, 0.9891859878671937, -0.14666656540331258], [0.10084999950729188, -0.014243196761691612, 0.22450448243478027])


In [65]:
#TODO: your visual servoing code here

#try to keep the object 30cm away from the sensor, in the center of the image
desiredObjectCameraShift = [0,0,0.3]

#you can put other state variables here
numLoops = 0

def visualServoInit(robot,q):
    """Initialize the controller.  This is called at the start of each simulation."""
    global numLoops
    #place whatever code you want here
    numLoops = 0

def visualServoFixedOrientation(robot,dt,q,sensorReading):
    """Takes the current sensed q and sensor reading, and outputs a desired position command for the robot.
    
    This is run every dt seconds.
    
    Do not "cheat" and use the object position directly!
    """
    global numLoops
    
    numLoops += 1
    if sensorReading is None:
        #didn't see the object, do nothing
        return q
    #provided code from HW4
    T = vectorops.mul(vectorops.sub(desiredObjectCameraShift,sensorReading[1]),0.1)
    camWorldXform = se3.mul(cameraLink.getTransform(),cameraXform)
    #convert the error into link local coordinates
    eR = [0,0,0]   #rotation error
    et = so3.apply(camWorldXform[0],T)
    robot.setConfig(q)
    J = cameraLink.getJacobian((0,0,0))
    #jacobian has orientation rows on top, translation rows on bottom
    #several joints are not allowed to move, so zero those out
    qmin,qmax = robot.getJointLimits()
    for j in range(cameraLink.index+1):
        if qmin[j] == qmax[j]:
            for i in range(6):
                J[i][j] = 0.0
    err = list(eR) + list(et)
    #pseudoinverse method
    dq = np.dot(np.linalg.pinv(J),err)
    return vectorops.madd(q,dq,-1.0)

def visualServoFreeOrientation(robot,dt,q,sensorReading):
    """Takes the current sensed q and sensor reading, and outputs a desired position command for the robot.
    
    This is run every dt seconds.
    
    Do not "cheat" and use the object position directly!
    """
    global numLoops
    
    numLoops += 1
    if sensorReading is None:
        #didn't see the object, do nothing
        return q
    #do nothing? TODO
    return q

def visualServoMatchOrientation(robot,dt,q,sensorReading):
    """Takes the current sensed q and sensor reading, and outputs a desired position command for the robot.
    
    This is run every dt seconds.
    
    Do not "cheat" and use the object position directly!
    """
    global numLoops
    
    numLoops += 1
    if sensorReading is None:
        #didn't see the object, do nothing
        return q
    #do nothing? TODO
    return q

## Object movement models

There are several object movement models available for you to test.  When you are developing and testing your methods, use the options below to examine how your controller behaves in response to certain object behaviors.

In [74]:
def objectMovement(t):
    """Returns w,v where w is the object's angular velocity and v is the translational velocity"""
    #FOR TESTING: change the object behavior here
    #stationary
    return [0,0,0],[0,0,0]
    
    #step change at t=2
    #if t >= 1.95 and t < 2.05:
    #    return [0,0,0],[0,-1,0]
    #else:
    #    return [0,0,0],[0,0,0]
    
    #moving back and forth
    #return [0,0,0],[math.cos(t*0.5)*0.02,0,0]
    
    #moving back and forth quickly
    #return [0,0,0],[math.cos(t*2)*0.08,0,0]
    
    #moving side to side with a moderate amplitude
    #return [0,0,0],[0,math.cos(t*2)*0.3,0]
    
    #moving side to side with a large amplitude
    #return [0,0,0],[0,math.cos(t*2)*0.6,0]
    
    #rotating in place
    #return [0,math.cos(t)*0.2,0],[0,0,0]
    
    #A different rotation angle
    #return [math.cos(t)*0.2,0,0],[0,0,0]
    
    #moving into view (use this when starting at the out-of-FOV location
    #return [0,math.cos(t)*0.2,0],[0,math.exp(-t*2),0]
    
    #moving in and out of view 
    #w = [0,math.cos(t)*0.2,0]
    #u = t % 4
    #if u >= 1.95 and u <= 2.05:
    #    return [5,w[1],w[2]],[0,10,0]
    #elif u >= 2.45 and u <= 2.55:
    #    return w,[0,-10,0]
    #return w,[0,0,0]
    
    #moving noisily
    #rvel = 0.02
    #tvel = 0.05
    #return [random.gauss(0,rvel) for i in range(3)],[random.gauss(0,tvel) for i in range(3)]


In [75]:
#test your code by running this cell
obj.setTransform(*Tobj_orig)
setConfig(qhome)
#initialize the controller
visualServoInit(robot,qhome)

print "Scroll up to look at the visualization... starting in 2s"
time.sleep(2.0)  #this gives you time to scroll up to watch the movement
print "Starting simulation now"
dt = 0.1 
t = 0
for i in xrange(100):  
    #simulate the sensor and controller
    s = emulateSensor()
    q = robot.getConfig()
    
    #FOR TESTING: choose your controller here
    qnext = visualServoFixedOrientation(robot,dt,q,s)
    #qnext = visualServoFreeOrientation(robot,dt,q,s)
    #qnext = visualServoMatchOrientation(robot,dt,q,s)
    
    #simulate the object movement
    T = obj.getTransform()
    w,v = objectMovement(t)
    rot = so3.mul(so3.from_rotation_vector(vectorops.mul(w,dt)),T[0])
    pos = vectorops.madd(T[1],v,dt)
    obj.setTransform(rot,pos)
    
    dq = vectorops.div(vectorops.sub(qnext,q),dt)
    vmax = robot.getVelocityLimits()
    for i in range(len(vmax)):
        if abs(dq[i]) > vmax[i]+1e-10:
            print "VELOCITY LIMIT",i,"VIOLATED",dq[i],">",vmax[i]
    setConfig(qnext)
    t += dt
    kvis.update()
    time.sleep(0.1)
print "Stopping simulation"

Scroll up to look at the visualization... starting in 2s
Starting simulation now
Stopping simulation


## Assignment

Your goal is to implement and study control strategies for the following:

1. Modify the provided controller to estimate the the velocity of the object, and incorporate this into the visualServoFixedOrientation control loop so that the robot matches the object's velocity better. 

2. For large magnitude movements of the object, the fixed orientation constraint is a big limitation.  Implement visualServoFreeOrientation so that the camera link is not required to stay at the same orientation.  

3. In visualServoMatchOrientation implement a controller that, when it first sees an object, remembers the object's relative orientation.  Thereafter, it matches the camera's orientation to maintain the same orientation, so if the object tilts, the camera should tilt to match.  (If the object disappears and reappears, it should match the object's new orientation)

In each of these implementations, the robot's joint positions and velocities should not exceed their limits, and the jerkiness of the robot should not be severe (visually obvious).  Don't worry about self-collision or collision with the table.

In your report, precisely and in technical English language (not code, although pseudocode may be acceptable), describe your strategy for implementing each controller, including but not limited to:

* The mathematical formulation of each control strategy

* The mathematical formulation of the object velocity estimation strategy

* A mathematical formulation of how you handle null-space motions with the free-orientation controller.

* A state diagram of the orientation-matching controller

* Your choice of gain constants and a rationale for choosing these (e.g., did you perform empirical tuning, use intuition, or is there a theoretical justification?)

## Unit testing

Perform unit testing of your controller, and describe the performance metrics and test procedures that you are using.  Interpret the observations you have made, including but not limited to:

* Whether performance is sensitive to certain parameters.
* Whether performance limitations or bottlenecks can be identified.
* Whether unit tests agree with theoretical behavior.

Parts of the provided code marked FOR_TESTING should be modified and tuned to provide these results. Also, you might find it useful to develop auxiliary testing code that measures performance and generates data for different parameters.

Specifically, you should at least:

* Test Controller 1 with object motions of different velocities.  Test how it responds to step functions and noise in the object movement. Test how it responds to noise to the sensor measurements (see the sensor emulation cell above).

* Test Controller 1 and 2 with large magnitude movements, and examine how well the controllers are able to follow the object movement.

* Test Controller 3 with varying object orientations, including the case in which the object appears into the field of view, or moves in and out.